In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re

In [5]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('2021 END OF YEAR ROUND-UP_ South Florida Office Sales - FINAL.csv')

In [6]:
df

,#,Property Name,Property Address,City,County,Class,Size (SF),Sale Price,Price/SF,Sale Date,Buyer,Seller,Clip,Source(s),NOTES
0,1,Phillips Point,777 S Flagler Dr & 201 Lakeview Ave,West Palm Beach,Palm Beach,A,"451,858","$281,850,000",$ 623.76,Q1 2021,The Related Companies,AEW Capital Management,https://therealdeal.com/miami/2021/01/22/relat...,"Collier's, CBRE",NaN
1,2,Citigroup Center,201 S Biscayne Blvd,Miami,Miami-Dade,A,"812,525","$270,000,000",$ 332.30,Q2 2021,JV: Monarch Alternative Capital / Tourmaline C...,CP Group,https://therealdeal.com/miami/2021/05/12/monar...,"Collier's, CBRE",Price is story is higher than Collier's reports
2,3,2 & 3 MiamiCentral,161 NW 6th St & 700 NW 1st St,Miami,Miami-Dade,A,"320,000","$230,000,000",$ 718.75,Q1 2021,Blackstone Real Estate,Shorenstein Properties LLC,https://www.miamiherald.com/news/business/arti...,Collier's,NaN
3,4,One Biscayne Tower,2 S Biscayne Blvd,Miami,Miami-Dade,A,"691,783","$225,000,000",$ 325.25,Q2 2021,JV: CP Group / Rialto Capital Management / Sab...,Dow Chemical Company,https://therealdeal.com/miami/2021/06/30/jv-bu...,"Collier's, CBRE",NaN
4,5,CityPlace Tower,525 Okeechobee Blvd,West Palm Beach,Palm Beach,A,"306,007","$175,000,000",$ 571.88,Q2 2021,Related Companies,W.R. Berkley Corporation,https://therealdeal.com/miami/2021/05/24/relat...,"Collier's, CBRE",NaN
5,6,Aventura Corporate Center,"20801, 20803, 20807 Biscayne Blvd",Aventura,Miami-Dade,A,"251,773","$140,000,000",$ 556.06,Q3 2021,Stonecutter Capital Management,Renaissance Properties,https://therealdeal.com/miami/2021/07/28/renai...,Collier's,NaN
6,7,Waterford Atrium,"6100, 6303, 6505 Blue Lagoon Dr",Miami,Miami-Dade,A,"495,485","$110,140,100",$ 222.29,Q2 2021,RealOp Investments,MetLife Real Estate Investments,https://therealdeal.com/miami/2021/05/04/realo...,Collier's,NaN
7,8,One Town Center,1 Town Center Rd,Boca Raton,Palm Beach,A,"191,294","$99,500,000",$ 520.14,Q3 2021,Prime US REIT,JV: Crocker Partners / Siguler Guff,https://therealdeal.com/miami/2021/07/20/singa...,"Collier's, CBRE",NaN
8,9,Doral Concourse,8400 NW 36th St,Miami,Miami-Dade,A,"240,669","$96,000,000",$ 398.89,Q3 2021,MG3 Developer Group,DRA Advisors,https://therealdeal.com/miami/2021/07/14/mg3-g...,"Collier's, CBRE",NaN
9,10,Aventura Harbour Centre (10X Centre),18851 NE 29th Ave,Aventura,Miami-Dade,A,"219,000","$93,000,000",$ 415.53,Q2 2021,Cardone Capital,C-III Capital Partners LLC,https://therealdeal.com/miami/2021/06/01/grant...,"Collier's, CBRE",NaN


In [7]:
df['geo_address'] = df['Property Address']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [10]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [11]:
df['loc']

0    (201 Lakeview Ave, West Palm Beach, FL 33401, ...
1    (201 S Biscayne Blvd, Miami, FL 33132, USA, (2...
2    (161 NW 6 St, Miami, FL 33128, USA, (25.780327...
3    (2 S Biscayne Blvd, Miami, FL 33131, USA, (25....
4    (525 Okeechobee Blvd, West Palm Beach, FL 3340...
5    (20801 Biscayne Blvd, Miami, FL 33180, USA, (2...
6    (6303 Blue Lagoon Dr #6100, Miami, FL 33126, U...
7    (1 Town Center Rd, Boca Raton, FL 33486, USA, ...
8    (8400 NW 36th St, Doral, FL 33166, USA, (25.80...
9    (18851 NE 29th Ave, Aventura, FL 33180, USA, (...
Name: loc, dtype: object

In [12]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [13]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,#,Property Name,Property Address,City,County,Class,Size (SF),Sale Price,Price/SF,Sale Date,...,Seller,Clip,Source(s),NOTES,geo_address,loc,point,lat,lon,altitude
0,1,Phillips Point,777 S Flagler Dr & 201 Lakeview Ave,West Palm Beach,Palm Beach,A,"451,858","$281,850,000",$ 623.76,Q1 2021,...,AEW Capital Management,https://therealdeal.com/miami/2021/01/22/relat...,"Collier's, CBRE",NaN,777 S Flagler Dr & 201 Lakeview Ave,"(201 Lakeview Ave, West Palm Beach, FL 33401, ...","(26.7066782, -80.05103799999999, 0.0)",26.706678,-80.051038,0.0
1,2,Citigroup Center,201 S Biscayne Blvd,Miami,Miami-Dade,A,"812,525","$270,000,000",$ 332.30,Q2 2021,...,CP Group,https://therealdeal.com/miami/2021/05/12/monar...,"Collier's, CBRE",Price is story is higher than Collier's reports,201 S Biscayne Blvd,"(201 S Biscayne Blvd, Miami, FL 33132, USA, (2...","(25.772249, -80.18677389999999, 0.0)",25.772249,-80.186774,0.0
2,3,2 & 3 MiamiCentral,161 NW 6th St & 700 NW 1st St,Miami,Miami-Dade,A,"320,000","$230,000,000",$ 718.75,Q1 2021,...,Shorenstein Properties LLC,https://www.miamiherald.com/news/business/arti...,Collier's,NaN,161 NW 6th St & 700 NW 1st St,"(161 NW 6 St, Miami, FL 33128, USA, (25.780327...","(25.7803273, -80.19735659999999, 0.0)",25.780327,-80.197357,0.0
3,4,One Biscayne Tower,2 S Biscayne Blvd,Miami,Miami-Dade,A,"691,783","$225,000,000",$ 325.25,Q2 2021,...,Dow Chemical Company,https://therealdeal.com/miami/2021/06/30/jv-bu...,"Collier's, CBRE",NaN,2 S Biscayne Blvd,"(2 S Biscayne Blvd, Miami, FL 33131, USA, (25....","(25.7737878, -80.1879712, 0.0)",25.773788,-80.187971,0.0
4,5,CityPlace Tower,525 Okeechobee Blvd,West Palm Beach,Palm Beach,A,"306,007","$175,000,000",$ 571.88,Q2 2021,...,W.R. Berkley Corporation,https://therealdeal.com/miami/2021/05/24/relat...,"Collier's, CBRE",NaN,525 Okeechobee Blvd,"(525 Okeechobee Blvd, West Palm Beach, FL 3340...","(26.7066126, -80.0559808, 0.0)",26.706613,-80.055981,0.0
5,6,Aventura Corporate Center,"20801, 20803, 20807 Biscayne Blvd",Aventura,Miami-Dade,A,"251,773","$140,000,000",$ 556.06,Q3 2021,...,Renaissance Properties,https://therealdeal.com/miami/2021/07/28/renai...,Collier's,NaN,"20801, 20803, 20807 Biscayne Blvd","(20801 Biscayne Blvd, Miami, FL 33180, USA, (2...","(25.9679722, -80.1437527, 0.0)",25.967972,-80.143753,0.0
6,7,Waterford Atrium,"6100, 6303, 6505 Blue Lagoon Dr",Miami,Miami-Dade,A,"495,485","$110,140,100",$ 222.29,Q2 2021,...,MetLife Real Estate Investments,https://therealdeal.com/miami/2021/05/04/realo...,Collier's,NaN,"6100, 6303, 6505 Blue Lagoon Dr","(6303 Blue Lagoon Dr #6100, Miami, FL 33126, U...","(25.7835999, -80.3005047, 0.0)",25.783600,-80.300505,0.0
7,8,One Town Center,1 Town Center Rd,Boca Raton,Palm Beach,A,"191,294","$99,500,000",$ 520.14,Q3 2021,...,JV: Crocker Partners / Siguler Guff,https://therealdeal.com/miami/2021/07/20/singa...,"Collier's, CBRE",NaN,1 Town Center Rd,"(1 Town Center Rd, Boca Raton, FL 33486, USA, ...","(26.3619507, -80.1284648, 0.0)",26.361951,-80.128465,0.0
8,9,Doral Concourse,8400 NW 36th St,Miami,Miami-Dade,A,"240,669","$96,000,000",$ 398.89,Q3 2021,...,DRA Advisors,https://therealdeal.com/miami/2021/07/14/mg3-g...,"Collier's, CBRE",NaN,8400 NW 36th St,"(8400 NW 36th St, Doral, FL 33166, USA, (25.80...","(25.8090526, -80.33283879999999, 0.0)",25.809053,-80.332839,0.0
9,10,Aventura Harbour Centre (10X Centre),18851 NE 29th Ave,Aventura,Miami-Dade,A,"219,000","$93,000,000",$ 415.53,Q2 2021,...,C-III Capital Partners LLC,https://therealdeal.com/miami/2021/06/01/grant...,"Collier's, CBRE",NaN,18851 NE 29th Ave,"(18851 NE 29th Ave, Aventura, FL 33180, USA, (...","(25.9501278, -80.1417244, 0.0)",25.950128,-80.141724,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [29]:
df = df.rename(columns={' Size (SF) ':'Square Footage','Sale Price ':'Sale Price',' Price/SF ':'Price/SF'})
df.columns

Index(['#', 'Property Name', 'Property Address', 'City', 'County', 'Class',
       'Square Footage', 'Sale Price', 'Price/SF', 'Sale Date', 'Buyer',
       'Seller', 'Clip', 'Source(s)', 'NOTES', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [32]:
df['Price/SF'] = df['Price/SF'].replace(' ','',regex=True)

In [41]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9)
# folium.TileLayer('stamentoner').add_to(m)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'2021 South Florida Priciest Office Sales')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('The most expensive office sales in South Florida')


marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Property Name"], r["Property Address"], r["City"], r["County"], r["Square Footage"], r["Sale Price"], r["Price/SF"], r["Sale Date"], r["Buyer"], r["Seller"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Property Name: {bold_start}{info[0]}{bold_end}{new_line}\
    Property Address: {bold_start}{info[1]}{bold_end}{new_line}\
    Buyer: {bold_start}{info[8]}{bold_end}{new_line}\
    Seller: {bold_start}{info[9]}{bold_end}{new_line}\
    City: {bold_start}{info[2]}{bold_end}{new_line}\
    County: {bold_start}{info[3]}{bold_end}{new_line}\
    Square Footage: {bold_start}{info[4]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[5]}{bold_end}{new_line}\
    Price/SF: {bold_start}{info[6]}{bold_end}{new_line}\
    Sale Date: {bold_start}{info[7]}{bold_end}{new_line}'\
    
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [42]:
m.save('index.html')